In [1]:
import math
import numpy as np
import seaborn as sns
from time import time
import pandas as pd
import pymcmcstat
import matplotlib.pyplot as plt
from pymcmcstat.MCMC import MCMC
from pymcmcstat.MCMC import DataStructure,  ModelParameters

from ipywidgets import interact
import ast

### Define model functions

In [2]:
def model_magnitud(theta, data):
    mag_v_sun = -26.72

    phase_angle = data['obs_phase_angle']
    range_sat = data['range_sat']
    cross_section = theta['cross_section']
    albedo = theta['albedo'] 
    mix_coef = theta['mix_coeff']

   # Change degrees to radian
    degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
    coe_1 = 2/(3*np.pi)
    coe_2 = 1/(4*np.pi)

    phase_angle_rad = degtorad(phase_angle)	

    part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
	
    mag = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coef * part1) + coe_2) + 5*np.log10(range_sat)
    return mag.to_numpy()

### Define sum-of-square function

In [3]:
def sum_squares(theta, data):

    rest = data[3].to_numpy() - model_magnitud(theta, data)
    print(rest)
    return rest #(rest ** 2).sum(axis=0)

def residual_calc(theta, data_x, data_y):
    residual = data_y - model_magnitud(theta, data_x)
    return residual

### Read Data and create new dataframe

In [4]:

path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
data_sat = pd.read_csv(path)
# data_sat.columns

data_in = pd.DataFrame()
data_in['obs_phase_angle'] = data_sat['obs_phase_angle']
data_in['range_sat'] = data_sat['range_sat']
data_in['mag_observation'] = data_sat['mag_observation']

### Setup Model Parameters

theta0: Initial parameter values
bounds: Min/Max
names: Reference names (keys)
longnames: LaTeX style names for plotting

In [5]:
theta0 = dict(
        cross_section=0.5,
        albedo=0.,
        mix_coef=0.5
        )
theta0vec = list(theta0.values())
bounds = dict(
        cross_section=[0.5, 1.3],
        albedo=[0., 1.],
        mix_coef=[0., 1.]
        )
names = ['cross_section', 'albedo', 'mix_coef']

### Define data for pymcmcstat.MCMC

In [6]:
data = DataStructure()
data.add_data_set(x=data_in, y=data_in['mag_observation'])

### Specify Parameters to Sample
sample: Flag to include parameter in MCMC sampling

In [ ]:
def g(cross_section, albedo, mix_coef):
    return dict(
        cross_section=cross_section,
        albedo=albedo,
        mix_coef=mix_coef)

select_sample = interact(g, cross_section=True, albedo=True, mix_coef=True)
sample_output = select_sample.widget.out
print

### Setup & Run MCMC Simulation

##### Setup:

- Add data
- Add model parameters
- Define model settings include sum-of-squares function
- Define simulation options

Run and display chain statistics. We remove the first part of chain as the burnin period.

#### - Add data

In [7]:
# Initialize MCMC object
mcstat = MCMC()
# Add data
mcstat.data = data

#### - Add model parameters

In [9]:
def ssfun(t, data):
    # Unpack data structure
    stress = data.ydata[0]
    # Evaluate model
    stress_model = model_magnitud(t, data)
    # Calculate sum-of-squares error
    res = stress.reshape(stress_model.shape) - stress_model
    ss = np.dot(res.T, res)
    return ss

In [10]:
# define model parameters
# sample = ast.literal_eval(sample_output.outputs[0]['data']['text/plain'])
for ii, name in enumerate(names):
    print(names[ii])
    mcstat.parameters.add_model_parameter(
            name=names[ii],
            theta0=theta0[name],
            minimum=bounds[name][0],
            maximum=bounds[name][1],
            sample=True)
mcstat.model_settings.define_model_settings(sos_function = ssfun, model_function=model_magnitud)

cross_section
albedo
mix_coef


In [16]:
mcstat.model_settings.display_model_settings(print_these=None)

model settings:
	sos_function = <function ssfun at 0x7f38d3ba41f0>
	model_function = <function model_magnitud at 0x7f38d674c280>
	sigma2 = None
	N = None
	N0 = None
	S20 = [nan]


AttributeError: 'ModelSettings' object has no attribute 'nsos'

#### - Define model settings include sum-of-squares function

In [ ]:
mcstat.simulation_options.define_simulation_options(
    nsimu=5.0e3,
    updatesigma=True)

#### - Run Simulation

In [ ]:
mcstat.run_simulation()

#### - View results

In [ ]:
results = mcstat.simulation_results.results
plotnames = results['names']
fullchain = results['chain']
fulls2chain = results['s2chain']
nsimu = results['nsimu']
burnin = int(nsimu/2)
chain = fullchain[burnin:, :]
s2chain = fulls2chain[burnin:, :]

mcstat.chainstats(chain, results)